In [74]:
import numpy as np
import scipy.signal as sig
import matplotlib.pyplot as plt

%matplotlib widget
# %matplotlib inline

import ipywidgets as widgets

%config InlineBackend.figure_format = 'retina'

### Linear filter model:

#### You leaned that we can represent a neuron by a basic model of a linear filter on the stimulus: 
#### $ r_{est}(t) = r_0 + \int_{0}^{\infty}D(\tau)s(t-\tau)d\tau  $
#### $ r_{0} $ - baseline firing rate, D - response kernel, $ r_{est}(t) $ - firing rate function, s(t) - stimulus function

#### look at the following example and answer these questions:
#### 1) Describe in words what the kernel seems to be doing
#### 2) How do we calculate the estimate of the rate?
#### 3) What changes there are between the stimulus and the estimated rate fucntion?
#### 4) What are the differences between the rate estimate and the actual rate function?
#### 5) What can we do to correct the rate estimate?
#### 6) What causes the differences between the rate function and the rate as calculated from a spike train?



In [75]:
# generate stimulus:
samp = 1000
dt = 1/samp
stim_length_seconds = 3
stim_length = stim_length_seconds*samp #miliseconds
window = sig.gaussian(400,80)
base_rate = 10

stimulus = np.ones(stim_length)+base_rate
r = np.linspace(-np.pi*5,np.pi*5,int(stim_length*(2/3)))
r2 = np.linspace(-np.pi*20,np.pi*20,stim_length)
r3 = np.linspace(-np.pi*12,np.pi*12,int(stim_length*(2/3)))
r4 = np.linspace(-np.pi*220,np.pi*220,stim_length)

stimulus[int(stim_length*(1/3)):stim_length] += 30*np.sin(r)
stimulus += 8*np.sin(r2) + 3*np.sin(r4)
stimulus[:int(stim_length*(2/3))] += 15*np.sin(r3)
# display stimulus
fig, ax = plt.subplots(nrows=5, figsize=(8,8))
ax[0].plot(np.arange(0,stim_length_seconds,dt),stimulus)
ax[0].set_title('stimulus')
ax[0].set_xlabel('time(s)')
ax[0].set_ylabel('au')
# create kernel
kernel = np.arange(5,0,-0.02)
kernel = kernel/sum(kernel)
# display kernel
ax[1].plot(np.flip(kernel))
ax[1].set_title('kernel')
ax[1].set_xlabel('time(ms)')
# calculate rate estimate 
base_rate = 5
rate_estimate = base_rate + np.convolve(stimulus, kernel, 'same')
# display rate estimate
ax[2].plot(np.arange(0,stim_length_seconds,dt),rate_estimate)
ax[2].set_title('rate estimate')
ax[2].set_xlabel('time(s)')
ax[2].set_ylabel('spikes/s')
# get actual rate function
noise = np.random.normal(0,2,stim_length)
ratefunc = base_rate + np.convolve(stimulus, kernel, 'same') + noise
ratefunc[ratefunc<0] =0
# display rate function
ax[3].plot(np.arange(0,stim_length_seconds,dt),ratefunc)
ax[3].set_title('rate function')
ax[3].set_xlabel('time(s)')
ax[3].set_ylabel('spikes/s')
# create spike train based on the rate function
spike_train = (np.random.uniform(size=stim_length)<ratefunc/samp).astype(np.int32)
for i in range(3):
    spike_train = np.vstack((spike_train, (np.random.uniform(size=stim_length)<ratefunc/samp).astype(np.int32) ) )
response_summations = np.mean(spike_train, axis=0)
window = window/sum(window)/dt
rate = np.convolve(response_summations,window,'same')
ax[4].plot(np.arange(0,stim_length_seconds,dt),rate)
ax[4].set_title('rate calculated from spike train')
ax[4].set_xlabel('time(s)')
ax[4].set_ylabel('spikes/s')
fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
def calculate_STA(spike_train, stimulus, STAlength):
    spike_times = ### complete
    for spike in spike times:
        ### complete
    STA = ### complete
    return STA

fig, ax = plt.subplots(figsize = (6,3))
STA = calculate_STA(spike_train, stimulus, STAlength=250)
ax.plot(STA)



SyntaxError: invalid syntax (<ipython-input-3-303c465bd4fa>, line 2)

### Kernel estimation
#### The following piece of code estimates the kernel using white noise stimulus
#### Answer the following questions:
#### 1) What is the sequance of actions done to simulate the kernel?
#### 2) Is the estimation good? why?
#### 3) What changes between different runs of the same code? why? 
#### 3) What is this sutuation trying to simulate?
#### 4) How can this concept be applied?
#### Bonus: find the strange code piece

In [70]:
def calculate_STA(spike_train, stimulus, STAlength):
    STAmat = []
    for spike in np.nonzero(spike_train)[0]:
        if spike < STAlength:
            continue
        else:
            STAmat.append(stimulus[spike-STAlength:spike])
    STAmat = np.array(STAmat)
    return STAmat.mean(axis=0)

samp = 1000
duration = 100 # seconds
dt = 1/1000
kernel_size = 100
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(8,5), squeeze=False)

# create kernel
kernel = np.sin(np.linspace(0,np.pi*3,kernel_size))/2
# create white noise
white_noise = np.random.uniform(-1,1,samp*duration)

# calculate the rate function using kernel and white noise
rate_func = np.convolve(white_noise, np.flip(kernel),'full')
rate_func = rate_func[0:samp*duration] 
# generate spike train based on the rate function:
spike_array = (np.random.uniform(size=samp*duration)<rate_func).astype(np.int32)
# calculate spike triggered average
STA = calculate_STA(spike_array, white_noise, kernel_size)


ax[0,0].plot(kernel)
ax[0,0].set_title('original kernel')
ax[0,0].set_xlabel('time(ms)')
ax[0,1].plot(white_noise[0:1000])
ax[0,1].set_title('white noise stimulus')
ax[0,1].set_xlabel('time(ms)')
ax[1,0].plot(rate_func[0:1000])
ax[1,0].set_title('rate function')
ax[1,0].set_xlabel('time(ms)')
ax[1,1].plot(STA)
ax[1,1].set_title('estimated kernel')
ax[1,1].set_xlabel('time(ms)')
fig.tight_layout()

<ipython-input-70-d0beecde705a>:15: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(8,5), squeeze=False)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …